In [1]:
import numpy as np
from layers.layerBase import LayerBase
from layers.affine import Affine
from layers.batchNormalization import BatchNormalization
from layers.relu import Relu
from layers.dropout import Dropout
from functions.UtilityFunctions import scale
from models.model import Model
from models.trainer import Trainer
from models.optimizer import SGD, AdaGrad, Momentum, RMSprop, Adam
from functions.EvaluationFunctions import Metrics, TYPE
from functions.LossFunctions import linear

In [2]:
import pandas as pd
columns = {'CRIM':float, 'ZN':float, 'INDUS':float, 'CHAS':int, 'NOX':float, 'RM':float, 'AGE':float, 'DIS':float, 'RAD':object, 'TAX':int, 'PTRATIO':float, 'B':float, 'LSTAT':float, 'TARGET':float}
df = pd.read_csv('./housing.data', header=None, sep='\s+', na_values='na', names=columns.keys(), dtype=columns)
y = df['TARGET']
tp = df[['CHAS','RAD']]
df = df.drop(['TARGET','CHAS','RAD'], axis=1)
df = (df - df.mean()) / df.std(ddof=0)
tp = pd.get_dummies(tp, dummy_na=False, columns=['RAD'])
df = pd.concat([df,tp], axis=1, sort=False)
df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,TAX,PTRATIO,B,...,CHAS,RAD_1,RAD_2,RAD_24,RAD_3,RAD_4,RAD_5,RAD_6,RAD_7,RAD_8
0,-0.419782,0.284830,-1.287909,-0.144217,0.413672,-0.120013,0.140214,-0.666608,-1.459000,0.441052,...,0,1,0,0,0,0,0,0,0,0
1,-0.417339,-0.487722,-0.593381,-0.740262,0.194274,0.367166,0.557160,-0.987329,-0.303094,0.441052,...,0,0,1,0,0,0,0,0,0,0
2,-0.417342,-0.487722,-0.593381,-0.740262,1.282714,-0.265812,0.557160,-0.987329,-0.303094,0.396427,...,0,0,1,0,0,0,0,0,0,0
3,-0.416750,-0.487722,-1.306878,-0.835284,1.016303,-0.809889,1.077737,-1.106115,0.113032,0.416163,...,0,0,0,0,1,0,0,0,0,0
4,-0.412482,-0.487722,-1.306878,-0.835284,1.228577,-0.511180,1.077737,-1.106115,0.113032,0.441052,...,0,0,0,0,1,0,0,0,0,0


In [3]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(n_splits=1, train_size=0.7, random_state=0)
train_idx, test_idx = next(ss.split(df, y))
df = df.to_numpy()
y = y.to_numpy()
x_train, x_test, t_train, t_test = df[train_idx], df[test_idx], y[train_idx], y[test_idx]

In [4]:
#モデル生成
m = Model()
m.append(Affine(scale(21) * np.random.randn(21, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Dropout())
m.append(Affine(scale(100) * np.random.randn(100, 1), np.zeros(1)))
m.append(LayerBase())
m.summary()

--------------------------------------------------
layer : Affine
param : (21, 100)
param : (100,)
output: (100, 100)
layer : BatchNormalization
output: (100, 100)
layer : Relu
output: (100, 100)
layer : Dropout
output: (100, 100)
layer : Affine
param : (100, 100)
param : (100,)
output: (100, 100)
layer : BatchNormalization
output: (100, 100)
layer : Relu
output: (100, 100)
layer : Dropout
output: (100, 100)
layer : Affine
param : (100, 1)
param : (1,)
output: (100, 1)
layer : LayerBase
output: (100, 1)
--------------------------------------------------


In [5]:
#学習
t = Trainer(
    model=m,
    optimizer=AdaGrad(lr=0.1),
    loss=linear,
    metrics=Metrics(TYPE.R2)
)
t.fit(x_train, t_train, 20, 50)

loss      :390.39463 |r2        :-4.15859 
loss      :205.96042 |r2        :-1.25717 
loss      :125.43175 |r2        :-0.48841 
loss      :90.30602 |r2        :-0.12244 
loss      :65.71995 |r2        :0.05417 
loss      :56.36857 |r2        :0.23569 
loss      :50.56607 |r2        :0.42353 
loss      :44.22990 |r2        :0.46439 
loss      :42.11012 |r2        :0.47678 
loss      :30.18408 |r2        :0.60035 
loss      :38.13768 |r2        :0.56533 
loss      :26.96583 |r2        :0.64621 
loss      :25.95356 |r2        :0.63024 
loss      :41.85770 |r2        :0.51441 
loss      :30.67115 |r2        :0.58609 
loss      :28.70467 |r2        :0.60055 
loss      :34.52969 |r2        :0.59755 
loss      :28.93958 |r2        :0.67586 
loss      :36.94478 |r2        :0.57341 
loss      :32.73388 |r2        :0.55451 


In [6]:
#検証
t.predict(x_test, t_test)

loss      :26.05473 |r2        :-25.05473 


In [145]:
def add_metrics(mt, nm, *vals):
    if len(vals) == 1:
        m = mt.get(nm, [])
        m = np.append(m, vals).tolist()
    else:
        m = mt.get(nm, [[]] * len(vals))
        for i, v in enumerate(vals):
            m[i] = np.append(m[i], v).tolist()
    mt[nm] = m
    return mt

In [148]:
obs = 5#[1,2,3]
prd = 6#[2,3,4]
#mt = {}
add_metrics(mt, 'xyz', obs, prd)

{'xyz': [[1.0, 2.0, 3.0, 1.0, 2.0, 3.0, 5.0],
  [2.0, 3.0, 4.0, 2.0, 3.0, 4.0, 6.0]]}